In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [165]:
def round_to_nearest(value):
    if value % 25 == 0:
        return value
    elif value % 25 <= 12:
        return (value // 25) * 25
    else:
        return ((value // 25) + 1) * 25

In [ ]:
path = '/content/drive/My Drive/wild/training_data.csv'
data = pd.read_csv(path)
data

,0,latitude,longitude,brightness,scan,track,satellite,instrument,confidence,version,...,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant
0,0,13.0,-61.0,314.6,1.2,1.1,Aqua,MODIS,16,6.03,...,906.0,764.0,142.0,840.8,1272.1,893.9,753.8,140.1,840.8,1255.1
1,1,13.0,-61.0,320.4,1.3,1.1,Aqua,MODIS,59,6.03,...,816.0,663.0,153.0,718.1,1237.2,771.1,626.5,144.6,718.1,1169.1
2,2,13.0,-60.0,317.5,1.0,1.0,Terra,MODIS,66,6.03,...,726.0,601.0,125.0,798.0,1063.3,771.2,638.4,132.8,798.0,1129.5
3,4,13.0,-60.0,318.9,1.0,1.0,Terra,MODIS,70,6.03,...,748.0,539.0,209.0,651.8,1155.3,792.9,571.4,221.5,651.8,1224.7
4,5,13.0,-60.0,332.6,1.0,1.0,Terra,MODIS,85,6.03,...,629.0,415.0,214.0,539.9,1055.9,690.5,455.6,234.9,539.9,1159.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,10315,11.0,108.0,311.0,1.4,1.2,Aqua,MODIS,48,6.03,...,816.0,652.0,164.0,808.8,1139.4,788.4,630.0,158.5,808.8,1100.9
1541,10425,11.0,108.0,311.5,2.4,1.5,Aqua,MODIS,57,6.03,...,754.0,641.0,113.0,874.3,1036.4,696.7,592.3,104.4,874.3,957.6
1542,10515,11.0,108.0,314.8,1.4,1.2,Terra,MODIS,73,6.03,...,673.0,533.0,140.0,800.6,941.2,735.8,582.7,153.1,800.6,1029.0
1543,10655,11.0,108.0,324.4,1.0,1.0,Terra,MODIS,80,6.03,...,387.0,84.0,303.0,126.5,938.9,423.5,91.9,331.5,126.5,1027.4


In [169]:
import pandas as pd


confidence_mapping = {
    0: 'l',
    1: 'n',
    2: 'h'
}

d_n_mapping = {
    1: 'd',
    0: 'n',
}

data['confidence'] = data['confidence'].replace({v: k for k, v in confidence_mapping.items()})



data['daynight'] = data['daynight'].replace({v: k for k, v in d_n_mapping.items()})


df_numeric = data.select_dtypes(include=['float64', 'int64'])

df_numeric_no_duplicates = df_numeric.drop_duplicates()

df_numeric_no_duplicates_no_nan = df_numeric_no_duplicates.fillna(0)
df_numeric_no_duplicates_no_nan.shape
x = df_numeric_no_duplicates_no_nan.drop('confidence', axis=1)
y = df_numeric_no_duplicates_no_nan['confidence']
y = y.apply(round_to_nearest)
lat_long = ['latitude', 'longitude']
x_for_res =  x[lat_long]

needed_features = [
  'temperature_2m',
 'pressure_msl',
 'surface_pressure',
 'windspeed_10m',
 'winddirection_10m',
 'winddirection_100m',
 'windgusts_10m',
 ]

x = x[needed_features]




In [162]:
import pickle
from sklearn.ensemble import RandomForestClassifier

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
x_test = x.head(20)

In [170]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [10, 50, 100, 300],
    'max_depth': [None, 10, 20, 30]
}

rf_model = RandomForestClassifier()

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='accuracy', cv=5)

grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_

y_pred = best_rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Best model parameters:", grid_search.best_params_)
print("Model accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Best model parameters: {'max_depth': 10, 'n_estimators': 300}
Model accuracy: 0.6634304207119741
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
          25       0.60      0.17      0.27        35
          50       0.61      0.67      0.64       118
          75       0.68      0.77      0.72       131
         100       0.90      0.83      0.86        23

    accuracy                           0.66       309
   macro avg       0.56      0.49      0.50       309
weighted avg       0.66      0.66      0.64       309



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [173]:
file_path = '/content/drive/My Drive/wild/rf_model.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(best_rf_model, file)
print("Model saved successfully at:", file_path)

Model saved successfully at: /content/drive/My Drive/wild/rf_model.pkl


In [172]:
results = loaded_model.predict(x_test)
prediction_series = pd.Series(results, name='prediction')

result_dataset = pd.concat([x_for_res, prediction_series], axis=1)
print(result_dataset)

      latitude  longitude  prediction
0         13.0      -61.0        10.0
1         13.0      -61.0        50.0
2         13.0      -60.0        60.0
3         13.0      -60.0        70.0
4         13.0      -60.0        80.0
...        ...        ...         ...
1540      11.0      108.0         NaN
1541      11.0      108.0         NaN
1542      11.0      108.0         NaN
1543      11.0      108.0         NaN
1544      11.0      108.0         NaN

[1545 rows x 3 columns]
